In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
import sys

In [2]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: NVIDIA GeForce GTX 1650 with Max-Q Design


In [3]:
features = np.load("features.npy")
labels = np.load("labels.npy")

X_train, X_temp, y_train, y_temp = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_val   = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)
X_test  = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)

y_train = torch.tensor(y_train, dtype=torch.long)
y_val   = torch.tensor(y_val, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

BATCH_SIZE = 32

train_ds = TensorDataset(X_train, y_train)
val_ds   = TensorDataset(X_val, y_val)
test_ds  = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE)


In [4]:
class CNN_LSTM_Model_PyTorch(nn.Module):
    def __init__(self, num_classes=24):
        super().__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )

        # After 3 MaxPool2d(2), height & width are divided by 8
        self.lstm_input_size = 128 * ((150//8))  # height collapsed into features
        self.lstm = nn.LSTM(self.lstm_input_size, 128, batch_first=True, bidirectional=True)

        self.dropout1 = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128*2, 64)
        self.dropout2 = nn.Dropout(0.3)
        self.out = nn.Linear(64, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.cnn(x)  # (B, C, H, W)

        # Reshape to (B, time_steps, features)
        x = x.permute(0, 3, 1, 2)  # (B, W, C, H)
        x = x.contiguous().view(batch_size, x.size(1), -1)  # (B, W, C*H)

        lstm_out, (h_n, _) = self.lstm(x)
        h = torch.cat((h_n[-2], h_n[-1]), dim=1)  # (B, 256)

        h = self.dropout1(h)
        h = torch.relu(self.fc1(h))
        h = self.dropout2(h)
        out = self.out(h)

        return out


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM_Model_PyTorch().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.96)

In [6]:
writer = SummaryWriter(log_dir='./logs')

best_val_acc = 0.0

for epoch in range(15):
    model.train()
    train_loss, train_correct = 0.0, 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(output, dim=1)
        train_loss += loss.item() * X_batch.size(0)
        train_correct += (preds == y_batch).sum().item()
    
    scheduler.step()
    train_loss /= len(train_loader.dataset)
    train_acc = train_correct / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss, val_correct = 0.0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            output = model(X_batch)
            loss = criterion(output, y_batch)

            preds = torch.argmax(output, dim=1)
            val_loss += loss.item() * X_batch.size(0)
            val_correct += (preds == y_batch).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / len(val_loader.dataset)

    print(f"Epoch [{epoch+1}/15] | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    writer.add_scalar("Loss/Train", train_loss, epoch)
    writer.add_scalar("Loss/Val", val_loss, epoch)
    writer.add_scalar("Accuracy/Train", train_acc, epoch)
    writer.add_scalar("Accuracy/Val", val_acc, epoch)

    # checkpoint
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")



Epoch [1/15] | Train Loss: 3.0506 Acc: 0.0923 | Val Loss: 2.8517 Acc: 0.1909
Epoch [2/15] | Train Loss: 2.4667 Acc: 0.2626 | Val Loss: 1.9294 Acc: 0.4635
Epoch [3/15] | Train Loss: 1.7694 Acc: 0.4436 | Val Loss: 1.3661 Acc: 0.6463
Epoch [4/15] | Train Loss: 1.3537 Acc: 0.5790 | Val Loss: 1.0193 Acc: 0.7649
Epoch [5/15] | Train Loss: 1.0506 Acc: 0.6777 | Val Loss: 0.7567 Acc: 0.8433
Epoch [6/15] | Train Loss: 0.8324 Acc: 0.7554 | Val Loss: 0.5838 Acc: 0.8855
Epoch [7/15] | Train Loss: 0.6654 Acc: 0.8107 | Val Loss: 0.4353 Acc: 0.9183
Epoch [8/15] | Train Loss: 0.5242 Acc: 0.8611 | Val Loss: 0.3357 Acc: 0.9457
Epoch [9/15] | Train Loss: 0.4242 Acc: 0.8936 | Val Loss: 0.2622 Acc: 0.9585
Epoch [10/15] | Train Loss: 0.3371 Acc: 0.9235 | Val Loss: 0.2073 Acc: 0.9699
Epoch [11/15] | Train Loss: 0.2822 Acc: 0.9386 | Val Loss: 0.1613 Acc: 0.9719
Epoch [12/15] | Train Loss: 0.2286 Acc: 0.9545 | Val Loss: 0.1405 Acc: 0.9739
Epoch [13/15] | Train Loss: 0.1932 Acc: 0.9610 | Val Loss: 0.1108 Acc: 0.

In [ ]:
model.load_state_dict(torch.load("best_model.pt"))
model.eval()
test_correct = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        output = model(X_batch)
        preds = torch.argmax(output, dim=1)
        test_correct += (preds == y_batch).sum().item()

test_acc = test_correct / len(test_loader.dataset)
print(f"✅ Test Accuracy: {test_acc:.4f}")


/tmp/ipykernel_420/685210218.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))


✅ Test Accuracy: 0.9799
